In [82]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
import numpy as np
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split

%matplotlib inline
sns.set(style='whitegrid', palette='muted', font_scale=1.5)
rcParams['figure.figsize'] = 14, 8
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [83]:
churners=pd.read_csv('./Predict-Customer-Attrition-main/datasets/BankChurners.csv')

In [84]:
churners.columns

Index(['CLIENTNUM', 'Attrition_Flag', 'Customer_Age', 'Gender',
       'Dependent_count', 'Education_Level', 'Marital_Status',
       'Income_Category', 'Card_Category', 'Months_on_book',
       'Total_Relationship_Count', 'Months_Inactive_12_mon',
       'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal',
       'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt',
       'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio',
       'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1',
       'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2'],
      dtype='object')

In [85]:
list = ['Attrition_Flag', 'Total_Trans_Ct', 'Total_Trans_Amt', 'Total_Revolving_Bal', 'Total_Ct_Chng_Q4_Q1', 'Contacts_Count_12_mon', 'Total_Relationship_Count', 'Months_Inactive_12_mon', 'Months_on_book']
churners = churners[list]

In [86]:
object_columns = churners.select_dtypes('object').columns

for i in object_columns:

    lb = LabelEncoder()
    lb.fit(churners[i])
    churners[i] = lb.transform(churners[i])
    
    print(f'category : {np.unique(churners[i])}\nclasses : {lb.classes_}\n')

input = churners.iloc[:,1:]
target = churners.iloc[:,0]

category : [0 1]
classes : ['Attrited Customer' 'Existing Customer']



In [87]:
x_train, x_test, y_train, y_test = train_test_split(input, target, random_state=42, test_size=0.2)

ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_test = ss.fit_transform(x_test)


In [88]:
x_train.shape

(8101, 8)

In [89]:
#Sequential 객체에 layer add하기
model=keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[8,])) # 2이미지를 1D array로 변환
model.add(keras.layers.Dense(300,activation='relu'))
model.add(keras.layers.Dense(100,activation='relu'))
model.add(keras.layers.Dense(10,activation='softmax'))


In [90]:
model=keras.models.Sequential([
    keras.layers.Flatten(input_shape=[8,]),
    keras.layers.Dense(300,activation='relu'),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(100,activation='relu'),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(10,activation='softmax'),
    
    ])

In [91]:
model.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_14 (Flatten)         (None, 8)                 0         
_________________________________________________________________
dense_42 (Dense)             (None, 300)               2700      
_________________________________________________________________
dropout_4 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense_43 (Dense)             (None, 100)               30100     
_________________________________________________________________
dropout_5 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_44 (Dense)             (None, 10)                1010      
Total params: 33,810
Trainable params: 33,810
Non-trainable params: 0
_________________________________________________

In [92]:
#compile메서드를 통해 한번에 compile하기
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [93]:
loss=keras.losses.sparse_categorical_crossentropy
optimizer=keras.optimizers.SGD()
metrics=[keras.metrics.sparse_categorical_accuracy]

In [94]:
history=model.fit(x_train,y_train,epochs=30,
                  validation_data=(x_test,y_test))

Train on 8101 samples, validate on 2026 samples
Epoch 1/30
8101/8101 [==============================] - 1s 73us/sample - loss: 0.7919 - acc: 0.8277 - val_loss: 0.3815 - val_acc: 0.8736
Epoch 2/30
8101/8101 [==============================] - 0s 55us/sample - loss: 0.3293 - acc: 0.8880 - val_loss: 0.3003 - val_acc: 0.8850
Epoch 3/30
8101/8101 [==============================] - 0s 54us/sample - loss: 0.2856 - acc: 0.8966 - val_loss: 0.2786 - val_acc: 0.8889
Epoch 4/30
8101/8101 [==============================] - 0s 56us/sample - loss: 0.2663 - acc: 0.9019 - val_loss: 0.2629 - val_acc: 0.8939
Epoch 5/30
8101/8101 [==============================] - 0s 56us/sample - loss: 0.2539 - acc: 0.9026 - val_loss: 0.2526 - val_acc: 0.8968
Epoch 6/30
8101/8101 [==============================] - 0s 52us/sample - loss: 0.2441 - acc: 0.9038 - val_loss: 0.2400 - val_acc: 0.9057
Epoch 7/30
8101/8101 [==============================] - 0s 58us/sample - loss: 0.2367 - acc: 0.9075 - val_loss: 0.2324 - val_acc: 

In [95]:
def create_dataset(dataframe, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('Attrition_Flag')
  return tf.data.Dataset.from_tensor_slices((dict(dataframe), labels)) \
          .shuffle(buffer_size=len(dataframe)) \
          .batch(batch_size)

In [96]:
train, test = train_test_split(churners, test_size=0.2, random_state=RANDOM_SEED)

train_ds = create_dataset(train)
test_ds = create_dataset(test)